<a href="https://colab.research.google.com/github/Anhbt95/geemap/blob/main/3_using%20earth%20engine%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>